### Import library

In [13]:
from sklearn.model_selection import train_test_split
import import_ipynb
import mock_data
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

### Firebase access

In [15]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Path to your downloaded service account key
cred = credentials.Certificate('pmd-android-app-firebase-adminsdk-j4wdz-3c4dc5af47.json')

# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {'databaseURL': 'https://pmd-android-app-default-rtdb.firebaseio.com'})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [53]:
# Reference to your database path
ref = db.reference('/sensor_data/-NwHxhJA0nACvQulOv0H')

# Reading the data at the reference
data = ref.get()

# Printing the data retrieved
print(data)

{'Heart Rate': 68.0, 'Oxygen Level': 96.0, 'Perspiration': 158.2}


In [22]:
# Reference to your database path
ref = db.reference('/sensor_data')

# Reading the data at the reference
data = ref.get()

# Printing the data retrieved
print(data)

{'-NwHxhJA0nACvQulOv0H': {'Heart Rate': 68.0, 'Oxygen Level': 96.0, 'Perspiration': 158.2}, '-NwHxj2RbjJ5sll3LeKj': {'Heart Rate': 58.0, 'Oxygen Level': 95.6, 'Perspiration': 156.6}, '-NwHxmo7MwD9x8jxqeYQ': {'Heart Rate': 63.6, 'Oxygen Level': 92.0, 'Perspiration': 158.4}, '-NwHxqN3qYN9fhjEg-yX': {'Heart Rate': 66.0, 'Oxygen Level': 98.8, 'Perspiration': 155.4}, '-NwHxutpWdrdlMMK3Hq9': {'Heart Rate': 64.8, 'Oxygen Level': 98.6, 'Perspiration': 151.2}, '-NwHxy37GkQHnNIxH3vO': {'Heart Rate': 65.2, 'Oxygen Level': 98.4, 'Perspiration': 146.0}, '-NwHy-Witu2HNT_r86NV': {'Heart Rate': 66.6, 'Oxygen Level': 99.4, 'Perspiration': 146.8}, '-NwHy4NBV5OyiL7E8OJO': {'Heart Rate': 55.6, 'Oxygen Level': 99.2, 'Perspiration': 147.4}, '-NwHy5pgqDfdtgVZfGbB': {'Heart Rate': 53.0, 'Oxygen Level': 90.2, 'Perspiration': 146.0}, '-NwHy8xgdIt6Y6rIt4Xd': {'Heart Rate': 53.8, 'Oxygen Level': 99.4, 'Perspiration': 147.0}}


In [ ]:
# # Reference to your database
# ref = db.reference('data1')

# # Reading the data
# data = ref.get()
# print(data)

# # Writing data
# ref.set({
#     'data2': {
#         'bpm': 100,
#         'heart rate': 90,
#         'perspiration': 200,
#         'emotion': 'happy'
#         }
#         }
# )

# ref = db.reference('data2')

# # Reading the data
# data = ref.get()
# print(data)

# Mock data

### version 1. median

In [31]:
# Setting a random seed for reproducibility
np.random.seed(42)

# Generating synthetic data
num_samples = 1000

# Define parameters for each emotion state based on assumed physiological impacts
emotions = {
    'sad': {'perspiration_mean': 280-300, 'heart_rate_mean': 60-65, 'oxygen_level_mean': 90-96},
    'happy': {'perspiration_mean': 200-220, 'heart_rate_mean': 80-85, 'oxygen_level_mean': 96-99},
    'neutral': {'perspiration_mean': 250-260, 'heart_rate_mean': 65-70, 'oxygen_level_mean': 99-100},
    'anxious': {'perspiration_mean': 330-360, 'heart_rate_mean': 85-100, 'oxygen_level_mean': 85-95}
}

# Simulating data based on emotion-specific parameters
data = {
    'Perspiration': [],
    'Heart Rate': [],
    'Oxygen Level': [],
    'Emotion State': []
}

# Assign approximately equal numbers of samples per emotion
samples_per_emotion = num_samples // len(emotions)

for emotion, params in emotions.items():
    perspiration = np.random.normal(params['perspiration_mean'], 1, samples_per_emotion)
    heart_rates = np.random.normal(params['heart_rate_mean'], 5, samples_per_emotion)
    oxygen_levels = np.random.normal(params['oxygen_level_mean'], 1, samples_per_emotion)

    data['Perspiration'].extend(perspiration)
    data['Heart Rate'].extend(heart_rates)
    data['Oxygen Level'].extend(oxygen_levels)
    data['Emotion State'].extend([emotion] * samples_per_emotion)

# Ensure the total samples match the desired number in case of rounding issues
if len(data['Emotion State']) < num_samples:
    remaining_samples = num_samples - len(data['Emotion State'])
    for emotion in np.random.choice(list(emotions.keys()), remaining_samples):
        params = emotions[emotion]
        data['Perspiration'].append(np.random.normal(params['perspiration_mean'], 1))
        data['Heart Rate'].append(np.random.normal(params['heart_rate_mean'], 5))
        data['Oxygen Level'].append(np.random.normal(params['oxygen_level_mean'], 1))
        data['Emotion State'].append(emotion)

# Creating DataFrame
df = pd.DataFrame(data)

# Displaying the first few entries and statistical summary of the DataFrame
print(df.head())
print(df.describe())

   Perspiration  Heart Rate  Oxygen Level Emotion State
0    200.496714   50.695580     90.926178           sad
1    199.861736   61.589310     91.909417           sad
2    200.647689   67.610781     88.601432           sad
3    201.523030   62.162326     90.562969           sad
4    199.765847   49.403150     89.349357           sad
       Perspiration   Heart Rate  Oxygen Level
count   1000.000000  1000.000000   1000.000000
mean     252.553772    75.769930    102.488245
std       41.486602    17.315630      8.397422
min      197.380255    40.793663     87.303113
25%      221.645273    61.239919     95.878706
50%      249.855516    72.738936    104.884262
75%      281.743777    90.604593    109.986238
max      313.109919   114.926295    113.926238


### version 2. Making a range

In [18]:
import numpy as np
import pandas as pd

# Setting a random seed for reproducibility
np.random.seed(42)

# Generating synthetic data
num_samples = 1000

# Define parameters for each emotion state based on assumed physiological ranges
emotions = {
    'sad': {'perspiration_range': (280, 300), 'heart_rate_range': (60, 65), 'oxygen_level_range': (90, 96)},
    'happy': {'perspiration_range': (200, 220), 'heart_rate_range': (80, 85), 'oxygen_level_range': (96, 99)},
    'neutral': {'perspiration_range': (250, 260), 'heart_rate_range': (65, 70), 'oxygen_level_range': (99, 102)},
    'anxious': {'perspiration_range': (330, 400), 'heart_rate_range': (85, 100), 'oxygen_level_range': (85, 95)}
}


# Simulating data based on emotion-specific parameters
data = {
    'Perspiration': [],
    'Heart Rate': [],
    'Oxygen Level': [],
    'Emotion State': []
}

# Assign approximately equal numbers of samples per emotion
samples_per_emotion = num_samples // len(emotions)

for emotion, params in emotions.items():
    perspiration = np.random.uniform(
        params['perspiration_range'][0], params['perspiration_range'][1], samples_per_emotion)
    heart_rates = np.random.uniform(
        params['heart_rate_range'][0], params['heart_rate_range'][1], samples_per_emotion)
    oxygen_levels = np.random.uniform(
        params['oxygen_level_range'][0], params['oxygen_level_range'][1], samples_per_emotion)

    data['Perspiration'].extend(perspiration)
    data['Heart Rate'].extend(heart_rates)
    data['Oxygen Level'].extend(oxygen_levels)
    data['Emotion State'].extend([emotion] * samples_per_emotion)

# Ensure the total samples match the desired number in case of rounding issues
if len(data['Emotion State']) < num_samples:
    remaining_samples = num_samples - len(data['Emotion State'])
    for emotion in np.random.choice(list(emotions.keys()), remaining_samples):
        params = emotions[emotion]
        data['Perspiration'].append(np.random.uniform(
            params['perspiration_range'][0], params['perspiration_range'][1]))
        data['Heart Rate'].append(np.random.uniform(
            params['heart_rate_range'][0], params['heart_rate_range'][1]))
        data['Oxygen Level'].append(np.random.uniform(
            params['oxygen_level_range'][0], params['oxygen_level_range'][1]))
        data['Emotion State'].append(emotion)

# Creating DataFrame
df = pd.DataFrame(data)

# Displaying the first few entries and statistical summary of the DataFrame
print(df.head())
print(df.describe())

   Perspiration  Heart Rate  Oxygen Level Emotion State
0    287.490802   61.472244     94.188970           sad
1    299.014286   61.925489     93.216578           sad
2    294.639879   64.255683     91.857166           sad
3    291.973170   61.584610     94.882770           sad
4    283.120373   60.847464     94.108387           sad
       Perspiration   Heart Rate  Oxygen Level
count   1000.000000  1000.000000   1000.000000
mean     279.593115    76.277541     95.283915
std       57.521659    12.168301      4.394309
min      200.092640    60.054188     85.000116
25%      242.541093    65.003274     92.032655
50%      270.034100    75.011833     96.006563
75%      307.330693    84.992758     99.007984
max      399.569387    99.967313    101.986313


### version 3: Making a dataframe

In [28]:
df.head(-10)

,Perspiration,Heart Rate,Oxygen Level,Emotion State
0,287.490802,61.472244,94.188970,sad
1,299.014286,61.925489,93.216578,sad
2,294.639879,64.255683,91.857166,sad
3,291.973170,61.584610,94.882770,sad
4,283.120373,60.847464,94.108387,sad
...,...,...,...,...
985,398.669116,96.777843,94.670353,anxious
986,363.838521,99.869919,85.516687,anxious
987,390.448327,93.167235,90.047956,anxious
988,371.211207,99.441525,92.184539,anxious


In [16]:
# Convert DataFrame to dictionary suitable for Firebase
data_dict = df.to_dict(orient='records')  # Converts DataFrame to list of dicts
data_dict

[{'Perspiration': 158.2, 'Heart Rate': 68.0, 'Oxygen Level': 96.0},
 {'Perspiration': 156.6, 'Heart Rate': 58.0, 'Oxygen Level': 95.6},
 {'Perspiration': 158.4, 'Heart Rate': 63.6, 'Oxygen Level': 92.0},
 {'Perspiration': 155.4, 'Heart Rate': 66.0, 'Oxygen Level': 98.8},
 {'Perspiration': 151.2, 'Heart Rate': 64.8, 'Oxygen Level': 98.6},
 {'Perspiration': 146.0, 'Heart Rate': 65.2, 'Oxygen Level': 98.4},
 {'Perspiration': 146.8, 'Heart Rate': 66.6, 'Oxygen Level': 99.4},
 {'Perspiration': 147.4, 'Heart Rate': 55.6, 'Oxygen Level': 99.2},
 {'Perspiration': 146.0, 'Heart Rate': 53.0, 'Oxygen Level': 90.2},
 {'Perspiration': 147.0, 'Heart Rate': 53.8, 'Oxygen Level': 99.4},
 {'Perspiration': 264.2, 'Heart Rate': 67.0, 'Oxygen Level': 77.0},
 {'Perspiration': 267.0, 'Heart Rate': 64.6, 'Oxygen Level': 89.0},
 {'Perspiration': 269.2, 'Heart Rate': 62.2, 'Oxygen Level': 73.2},
 {'Perspiration': 263.6, 'Heart Rate': 59.0, 'Oxygen Level': 95.6},
 {'Perspiration': 266.4, 'Heart Rate': 54.2, 'Ox

### send back to Firebase

In [19]:
# Reference to the Firebase database path where data will be stored
ref = db.reference('mock_data')

# Pushing data to Firebase
for record in data_dict:
    ref.push(record)

print("Data has been pushed to Firebase.")


Data has been pushed to Firebase.


### Retrieve Data

In [34]:
# Reference to your database path
ref = db.reference('/mock_data')

# Reading the data at the reference
data = ref.get()

# Printing the data retrieved
print(data)

# df = pd.DataFrame.from_dict(data, orient='index')
# df.head(5)

{'-NwCtLHwUIv9xprbuz_e': {'Emotion State': 'sad', 'Heart Rate': 61.47224446034793, 'Oxygen Level': 94.18897028411847, 'Perspiration': 287.4908023769473}, '-NwCtLIx3lnNjfcqHgeK': {'Emotion State': 'sad', 'Heart Rate': 61.92548864300963, 'Oxygen Level': 93.21657819806472, 'Perspiration': 299.0142861281983}, '-NwCtLNIbsapm0PKyX1N': {'Emotion State': 'sad', 'Heart Rate': 64.25568335758429, 'Oxygen Level': 91.85716569771796, 'Perspiration': 294.6398788362281}, '-NwCtLO7snun-44Bkpwu': {'Emotion State': 'sad', 'Heart Rate': 61.58461002578139, 'Oxygen Level': 94.8827701182417, 'Perspiration': 291.97316968394074}, '-NwCtLOzMKSkkJpRwlxS': {'Emotion State': 'sad', 'Heart Rate': 60.84746373343046, 'Oxygen Level': 94.10838703532328, 'Perspiration': 283.12037280884874}, '-NwCtLQ9QsuA620Wa3zI': {'Emotion State': 'sad', 'Heart Rate': 62.78400631229175, 'Oxygen Level': 90.97570163606935, 'Perspiration': 283.11989040672404}, '-NwCtLR0M7jT58dZcAko': {'Emotion State': 'sad', 'Heart Rate': 64.6807738708039

# Train models

### 1. Decision Tree

In [35]:
class DecisionTreeModel:
    def __init__(self):
        self.model = DecisionTreeClassifier(random_state=42)
        self.scaler = StandardScaler()
        self.emotions = ['sad', 'happy', 'neutral', 'anxious']

    def prepare_data(self, df):
        X = df[['Perspiration', 'Heart Rate', 'Oxygen Level']]
        y = pd.Categorical(df['Emotion State']).codes
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42)
        self.X_train = self.scaler.fit_transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)

    def train(self):
        self.model.fit(self.X_train, self.y_train)

    def evaluate(self):
        predictions = self.model.predict(self.X_test)
        return accuracy_score(self.y_test, predictions)

    # def predict(self, new_data):
    #     new_df = pd.DataFrame([new_data])
    #     new_data_scaled = self.scaler.transform(new_data)
    #     prediction_index = self.model.predict(new_data_scaled)[0]
    #     return self.emotions[prediction_index]


    def predict(self, new_data):
        # Ensure new_data is a DataFrame with the correct columns
        if not isinstance(new_data, pd.DataFrame):
            # Converts a dict to DataFrame if necessary
            new_data = pd.DataFrame([new_data])

        # Scale the new data using the same scaler used during training
        new_data_scaled = self.scaler.transform(new_data)

        # Make predictions
        prediction_index = self.model.predict(new_data_scaled)[0]

        # Return the corresponding emotion
        return self.emotions[prediction_index]

In [36]:
# Reference to your database path
ref = db.reference('/mock_data')

# Reading the data at the reference
data = ref.get()

# Printing the data retrieved
print(data)

df = pd.DataFrame.from_dict(data, orient='index')

decision_tree_model = DecisionTreeModel()

decision_tree_model.prepare_data(df)
decision_tree_model.train()

print("Accuracy:", decision_tree_model.evaluate())

{'-NwCtLHwUIv9xprbuz_e': {'Emotion State': 'sad', 'Heart Rate': 61.47224446034793, 'Oxygen Level': 94.18897028411847, 'Perspiration': 287.4908023769473}, '-NwCtLIx3lnNjfcqHgeK': {'Emotion State': 'sad', 'Heart Rate': 61.92548864300963, 'Oxygen Level': 93.21657819806472, 'Perspiration': 299.0142861281983}, '-NwCtLNIbsapm0PKyX1N': {'Emotion State': 'sad', 'Heart Rate': 64.25568335758429, 'Oxygen Level': 91.85716569771796, 'Perspiration': 294.6398788362281}, '-NwCtLO7snun-44Bkpwu': {'Emotion State': 'sad', 'Heart Rate': 61.58461002578139, 'Oxygen Level': 94.8827701182417, 'Perspiration': 291.97316968394074}, '-NwCtLOzMKSkkJpRwlxS': {'Emotion State': 'sad', 'Heart Rate': 60.84746373343046, 'Oxygen Level': 94.10838703532328, 'Perspiration': 283.12037280884874}, '-NwCtLQ9QsuA620Wa3zI': {'Emotion State': 'sad', 'Heart Rate': 62.78400631229175, 'Oxygen Level': 90.97570163606935, 'Perspiration': 283.11989040672404}, '-NwCtLR0M7jT58dZcAko': {'Emotion State': 'sad', 'Heart Rate': 64.6807738708039

# Testing

In [37]:
# Example new data
# new_data = {'Perspiration': 250, 'Heart Rate': 70, 'Oxygen Level': 100}
# df = pd.DataFrame.from_records([new_data])

# Reference to your database path
ref = db.reference('/sensor_data')

# Reading the data at the reference
data = ref.get()

df = pd.DataFrame.from_dict(data, orient='index')

# Make sure the DataFrame has the same columns, in the same order as expected by the model
expected_columns = ['Perspiration', 'Heart Rate', 'Oxygen Level']
df = df[expected_columns]  # Reorder or select only the necessary columns

# print(df)

predicted_emotion = decision_tree_model.predict(df)
print("Decision Tree Prediction:", predicted_emotion)

Decision Tree Prediction: anxious


#### Sending the data to the firebase after testing

In [38]:
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime

In [39]:
from datetime import datetime

# Function to write emotion state result to Firebase
def write_emotion_result(emotion_result):
    # Create a reference to the emotions node in your database
    ref = db.reference('emotions_result')

    # Emotion to string mapping
    emotion_to_string = {
        'sad': """21 Breaths for Sadness:

- Sit or lie down comfortably with your eyes closed.
- Inhale slowly through your nose and exhale through your mouth.
- Focus on the sensation of your breath as it enters and exits your body.
- Imagine releasing tension with each exhale.
- Silently count to 21 deep breaths.
- Let go of any sadness or negativity as you continue to breathe deeply.

""",
        'happy': """Gratitude Focal Session for Happiness:

- Find a quiet, comfortable spot to sit or lie down.
- Close your eyes and breathe deeply to center yourself.
- Focus on something you’re grateful for—be it a person, place, or moment.
- Embrace the feeling of gratitude, noticing the positive emotions it evokes.
- Reflect briefly on your blessings, letting gratitude fill you.
- Open your eyes when ready and take this sense of gratitude with you through the day.

""",
        'neutral': """Neutral Emotion:

- Sit comfortably and focus on your breath, observing each inhalation and exhalation.
- Acknowledge any thoughts or sensations without judgment and return your attention to your breathing.
- Consider a body scan meditation, moving your awareness from your toes to the top of your head.
- Alternatively, try walking meditation, concentrating on the feel of each step.
- These practices aim to foster presence and inner peace, helping you fully engage with the present moment.

""",

        'anxious': """4-7-8 Meditation for Anxiety:

- Sit comfortably with a straight back and hands in your lap.
- Close your eyes, inhale through your nose for a count of 4, hold for 7, and exhale through your mouth for 8.
- Repeat this breathing cycle several times, focusing on the rhythm of your breath to foster calmness and grounding.
   
   """
    }

    # Get the corresponding string for the emotion result
    emotion_string = emotion_to_string.get(emotion_result, 'Unknown')

    # Use a timestamp as a unique key for each entry
    timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
    emotion_key = f"emotion_{timestamp}"

    # Write the data to Firebase under a unique key
    ref.child(emotion_key).set({
        'emotion_state': emotion_result,
        'emotion_code': emotion_string,  # Add the emotion string to Firebase
        'timestamp': timestamp
    })
    print(
        f"Data written under key: {emotion_key} with timestamp: {timestamp}, Emotion Code: {emotion_string}")

In [40]:
# Example use of the function
# Assume this is the result from your model
print(predicted_emotion)
write_emotion_result(predicted_emotion)

anxious
Data written under key: emotion_20240425152354 with timestamp: 20240425152354, Emotion Code: 4-7-8 Meditation for Anxiety:

- Sit comfortably with a straight back and hands in your lap.
- Close your eyes, inhale through your nose for a count of 4, hold for 7, and exhale through your mouth for 8.
- Repeat this breathing cycle several times, focusing on the rhythm of your breath to foster calmness and grounding.
   
   


### 2. Logical Regression

In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
class LogisticRegressionModel:
    def __init__(self):
        self.model = LogisticRegression(random_state=42)
        self.scaler = StandardScaler()

    def prepare_data(self, df):
        """
        Prepares training and testing data from the DataFrame.
        Assumes 'Emotion State' as the target and all other numerical columns as features.
        """
        X = df.drop('Emotion State', axis=1)
        y = pd.Categorical(df['Emotion State']).codes
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        self.X_train = self.scaler.fit_transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)

    def train(self):
        """Trains the Logistic Regression model."""
        self.model.fit(self.X_train, self.y_train)

    def evaluate(self):
        """Evaluates the Logistic Regression model using accuracy as the metric."""
        predictions = self.model.predict(self.X_test)
        return accuracy_score(self.y_test, predictions)

    def predict(self, new_data):
        """
        Makes predictions using the trained Logistic Regression model on new data.
        new_data should be a dictionary or a DataFrame row.
        """
        new_data_scaled = self.scaler.transform([list(new_data.values())])
        return self.model.predict(new_data_scaled)

In [ ]:
# Assuming 'df' is your DataFrame loaded with the appropriate data including 'Emotion State'
logistic_model = LogisticRegressionModel()
logistic_model.prepare_data(df)
logistic_model.train()
print("Logistic Regression Accuracy:", logistic_model.evaluate())

# Example new data
new_data = {'Perspiration': 250, 'Heart Rate': 70, 'Oxygen Level': 100}
print("Logistic Regression Prediction for new data:", logistic_model.predict(new_data))